In [1]:
import pandas as pd
from sqlalchemy import create_engine
from typing import Literal

# --- 1. DATABASE CONNECTION CONFIGURATION ---
# IMPORTANT: Replace 'user', 'password', and 'host' with your actual MySQL credentials.
DB_USER = 'root'
DB_PASSWORD = '5884123695476'
DB_HOST = 'localhost'
DB_NAME = 'sakila'

# Create the SQLAlchemy engine for database connection
try:
    engine = create_engine(f'mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}')
    print("Database connection engine created successfully.")
except ImportError:
    print("ERROR: Required libraries 'sqlalchemy', 'pandas', or 'pymysql' might be missing.")
    print("Please run: pip install sqlalchemy pandas pymysql")
    exit()

# ======================================================================
# 1. FUNCTION: rentals_month
# ======================================================================

def rentals_month(engine: create_engine, month: int, year: int) -> pd.DataFrame:
    """
    Retrieves rental data for a specific month and year from the Sakila 'rental' table.

    Args:
        engine: The SQLAlchemy engine object for the database connection.
        month: The month (integer) to filter the data.
        year: The year (integer) to filter the data.

    Returns:
        A pandas DataFrame containing the filtered rental data (rental_id, customer_id, rental_date).
    """
    # SQL query uses MONTH() and YEAR() functions to filter the rental_date column.
    query = f"""
    SELECT
        rental_id,
        customer_id,
        rental_date
    FROM
        rental
    WHERE
        YEAR(rental_date) = {year} AND MONTH(rental_date) = {month};
    """

    # Read the data directly into a Pandas DataFrame
    df_rentals = pd.read_sql(query, engine)
    
    return df_rentals

# ======================================================================
# 2. FUNCTION: rental_count_month
# ======================================================================

def rental_count_month(df_rentals: pd.DataFrame, month: int, year: int) -> pd.DataFrame:
    """
    Calculates the number of rentals made by each customer_id during the selected month and year.

    Args:
        df_rentals: DataFrame containing rental records (output from rentals_month).
        month: The month (integer).
        year: The year (integer).

    Returns:
        A new DataFrame with 'customer_id' and the rental count column, named 'rentals_MM_YYYY'.
    """
    
    # Group the data by 'customer_id' and count the number of rentals (rental_id)
    df_count = df_rentals.groupby('customer_id')['rental_id'].count().reset_index()
    
    # Format the month/year for the new column name (e.g., 5 -> '05')
    month_str = str(month).zfill(2)
    year_str = str(year)
    new_column_name = f"rentals_{month_str}_{year_str}"
    
    # Rename the rental_id count column to the required format
    df_count = df_count.rename(columns={'rental_id': new_column_name})
    
    return df_count

# ======================================================================
# 3. FUNCTION: compare_rentals
# ======================================================================

def compare_rentals(df1: pd.DataFrame, df2: pd.DataFrame) -> pd.DataFrame:
    """
    Compares the rental counts of two months per customer.

    Args:
        df1: DataFrame containing rental counts for the first month (e.g., May).
        df2: DataFrame containing rental counts for the second month (e.g., June).

    Returns:
        A combined DataFrame with an added 'difference' column (df2_rentals - df1_rentals).
    """
    
    # Identify the rental count column names dynamically (the second column in each DataFrame)
    col1 = df1.columns[1]
    col2 = df2.columns[1]

    # Merge the two DataFrames on 'customer_id' using an 'outer' join to keep all customers
    df_combined = pd.merge(df1, df2, on='customer_id', how='outer')
    
    # Fill NaN values with 0, as a NaN count means the customer made 0 rentals in that month.
    df_combined = df_combined.fillna(0)
    
    # Calculate the 'difference' column (Month 2 - Month 1)
    df_combined['difference'] = df_combined[col2] - df_combined[col1]
    
    return df_combined

# ======================================================================
# 4. SCRIPT EXECUTION
# ======================================================================

# Define the months of interest (May and June 2005)
MAY_MONTH = 5
MAY_YEAR = 2005
JUNE_MONTH = 6
JUNE_YEAR = 2005

print("\n--- Starting Data Retrieval and Analysis ---")

# Step 1: Retrieve raw data for May and June
df_may_raw = rentals_month(engine, MAY_MONTH, MAY_YEAR)
df_jun_raw = rentals_month(engine, JUNE_MONTH, JUNE_YEAR)
print(f"1. Raw rental data retrieved. May: {len(df_may_raw)} records. June: {len(df_jun_raw)} records.")


# Step 2: Count rentals per customer for each month
df_may_count = rental_count_month(df_may_raw, MAY_MONTH, MAY_YEAR)
df_jun_count = rental_count_month(df_jun_raw, JUNE_MONTH, JUNE_YEAR)
print(f"2. Rental counts calculated. Customers in May: {len(df_may_count)}. Customers in June: {len(df_jun_count)}.")


# Step 3: Compare and calculate the difference
df_final_report = compare_rentals(df_may_count, df_jun_count)


# Filter for customers who were active in BOTH May AND June
may_col_name = f"rentals_{str(MAY_MONTH).zfill(2)}_{MAY_YEAR}"
jun_col_name = f"rentals_{str(JUNE_MONTH).zfill(2)}_{JUNE_YEAR}"

df_active_both = df_final_report[
    (df_final_report[may_col_name] > 0) & 
    (df_final_report[jun_col_name] > 0)
].sort_values(by='difference', ascending=False)


print("\n--- FINAL CUSTOMER ACTIVITY REPORT (May vs. June) ---")
print(f"Total customers active in both months: {len(df_active_both)}")
print("\nTop 10 Customers by Rental Increase (June - May):")
print(df_active_both.head(10))
print("\nBottom 5 Customers by Rental Decrease (June - May):")
print(df_active_both.tail())

Database connection engine created successfully.

--- Starting Data Retrieval and Analysis ---
1. Raw rental data retrieved. May: 1156 records. June: 2311 records.
2. Rental counts calculated. Customers in May: 520. Customers in June: 590.

--- FINAL CUSTOMER ACTIVITY REPORT (May vs. June) ---
Total customers active in both months: 512

Top 10 Customers by Rental Increase (June - May):
     customer_id  rentals_05_2005  rentals_06_2005  difference
452          454              1.0             10.0         9.0
455          457              1.0              9.0         8.0
211          213              1.0              9.0         8.0
293          295              1.0              9.0         8.0
26            27              1.0              8.0         7.0
232          234              1.0              8.0         7.0
258          260              1.0              8.0         7.0
378          380              1.0              8.0         7.0
559          561              2.0           